In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

In [3]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [4]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

Iter 1/50 - Loss: 0.915   lengthscale: 0.693   noise: 0.693
Iter 2/50 - Loss: 0.883   lengthscale: 0.644   noise: 0.644
Iter 3/50 - Loss: 0.849   lengthscale: 0.598   noise: 0.598
Iter 4/50 - Loss: 0.811   lengthscale: 0.555   noise: 0.554
Iter 5/50 - Loss: 0.770   lengthscale: 0.514   noise: 0.513
Iter 6/50 - Loss: 0.724   lengthscale: 0.475   noise: 0.474
Iter 7/50 - Loss: 0.676   lengthscale: 0.439   noise: 0.437
Iter 8/50 - Loss: 0.627   lengthscale: 0.404   noise: 0.402
Iter 9/50 - Loss: 0.580   lengthscale: 0.371   noise: 0.370
Iter 10/50 - Loss: 0.538   lengthscale: 0.342   noise: 0.339
Iter 11/50 - Loss: 0.498   lengthscale: 0.315   noise: 0.311
Iter 12/50 - Loss: 0.462   lengthscale: 0.293   noise: 0.284
Iter 13/50 - Loss: 0.427   lengthscale: 0.274   noise: 0.260
Iter 14/50 - Loss: 0.392   lengthscale: 0.259   noise: 0.237
Iter 15/50 - Loss: 0.358   lengthscale: 0.247   noise: 0.216
Iter 16/50 - Loss: 0.324   lengthscale: 0.238   noise: 0.197
Iter 17/50 - Loss: 0.290   length

In [5]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = torch.linspace(0, 1, 51)
    observed_pred = likelihood(model(test_x))

In [6]:
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(4, 3))

    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])

: 